<a href="https://colab.research.google.com/github/olga-terekhova/pdf-utilities/blob/main/RotatePDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rotate PDF

## How to use

To **rotate** some pages in a PDF file once in a chosen direction:  
1) Prepare the PDF file that you want to rotate.  
2) Upload the pdf file that you want to rotate into the root directory of Files area. Upload one file only. E.g. *input.pdf*.  
3) In the [Set parameters](#scrollTo=XaMoALpy6JHx&line=7&uniqifier=1) section, set the preferred name for the rotated pdf. E.g. *output.pdf*.   
4) In the [Set parameters](#scrollTo=XaMoALpy6JHx&line=7&uniqifier=1) section, set *'rotate'* for the choice between *rotate* or *delete* (you may use the  dropdown field).  
5) In the [Set parameters](#scrollTo=XaMoALpy6JHx&line=7&uniqifier=1) section, set *'clockwise'* or *'counterclockwise'* for the choice of direction (you may use the  dropdown field).  
6) In the [Set parameters](#scrollTo=XaMoALpy6JHx&line=7&uniqifier=1) section, specify a range of pages to be rotated. You can use comma (,) for individual pages or dash (-) for ranges. Spaces are allowed but not needed. E.g. *'1,3'* or *'1,5-7,9'* or *'2, 4,5-10'*.     
7) Run all cells in the notebook (Runtime - Run all or Ctrl-F9).  
8) Download the output pdf from the Files area (Refresh to see the newly created rotated file).


If you need to rotate another file, **delete** current PDF files first.
For that:  
1) In the [Set parameters](#scrollTo=XaMoALpy6JHx&line=7&uniqifier=1) section, set 'delete' for the choice between rotate or delete (you may use the dropdown field).  
2) Run all cells in the notebook (Runtime - Run all or Ctrl-F9).  


If you need to **rotate** pages in the same file in **different** directions or to **rotate** them **twice**:  
1) Choose the direction for the first step and only select pages that need to be rotated in this direction.  
2) Run the rotation. For example, *page 1* in *input.pdf* will be rotated clockwise and the result will be saved in *step1.pdf*.  
3) In the Files area remove the original file (e.g. *input.pdf*) and set a new name for the output (e.g. *step2.pdf*). The output from the previous step (e.g. *step1.pdf*) will be a source for the next step.  
4) If you need to rotate pages for the second time in the same direction, repeat the same rotation. For example, *page 1* in *step1.df* will be rotated clockwise again and the result will be saved in *step2.pdf*.  
5) In the Files area remove the intermediary input file (e.g. *step1.pdf*) and set a new name for the output (e.g. *step3.pdf*). The output from the previous step (e.g. *step2.pdf*) will be a source for the next step.  
6) If you need to rotate pages in a different direction, change the direction parameter and only select pages that need to be rotated in this direction.  
7) Run the rotation. For example, *page 2* in *step2.pdf* will be rotated counter clockwise and the result will be stored in *step3.pdf*.  

In [35]:
# @title Set parameters

rotated_pdf_path = 'output.pdf' # @param {type:"string"}
rotate_or_delete = "rotate" # @param ["rotate", "delete"]
direction = "counterclockwise" # @param ["clockwise", "counterclockwise"]
page_range = "4" # @param {type:"string"}

print(rotated_pdf_path)
print(rotate_or_delete)
print(direction)
print(page_range)

output.pdf
delete
counterclockwise
4


## Code (you can collapse this section)

### Install, import, initialize  

In [36]:
!pip install -q PyPDF2

In [37]:
import os
import PyPDF2

### Rotate the PDF file

In [38]:
def get_file():
  """
  Get the first PDF file in the current directory.
  Return the file name and a message.
  """
  pdf_files = []
  for filename in os.listdir():
      if filename.endswith('.pdf'):
          pdf_files.append(filename)

  if rotated_pdf_path in pdf_files:
    return "", "File " + rotated_pdf_path + " already exists. No action taken. Do you want to delete PDF files first?"

  if len(pdf_files) == 0:
    return "", "No PDF files found. No action taken."

  # sort pdf_files in the alphabetical order
  pdf_files.sort()

  # take the first PDF file
  pdf_file = pdf_files[0]
  print(pdf_file)

  return pdf_file, "OK"

In [39]:
def parse_page_range(page_range):
    """
    Parse a string like '2, 5-7,9' into a list of page numbers.

    :param page_range: String representing the page range. Input by the user.
    Return a list of page numbers (0-based).
    """

    pages = set()

    # Remove all spaces from the input string
    page_range = page_range.replace(' ', '')

    # Split the string by commas
    ranges = page_range.split(',')

    for r in ranges:
        if '-' in r:
            start, end = map(int, r.split('-'))
            pages.update(range(start - 1, end))  # Convert to 0-based index
        else:
            pages.add(int(r) - 1)  # Convert to 0-based index


    return sorted(pages)

In [40]:
def rotate_selected_pages(output_pdf, page_range, direction):
    """
    Rotate selected pages in the PDF either 'clockwise' or 'counterclockwise'.

    :param output_pdf: Path to output PDF. Input by the user.
    :param page_range: Pages to rotate in string format (e.g., '2,5-7,9'). Input by the user.
    :param direction: Direction of rotation ('clockwise' or 'counterclockwise'). Input by the user.

    Return a message.
    """

    # Get the input PDF file (a first PDF file found in the root directory)
    input_pdf, input_pdf_response = get_file()
    if input_pdf == "": # no file to process
      print(input_pdf_response)
      return input_pdf_response

    # Parse the page range string
    pages_to_rotate = parse_page_range(page_range)

    # Open the PDF file
    with open(input_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        writer = PyPDF2.PdfWriter()

        # Set the rotation angle based on direction
        if direction == 'clockwise':
            angle = 90  # Clockwise rotation
        elif direction == 'counterclockwise':
            angle = -90  # Counterclockwise rotation
        else:
            raise ValueError("Direction must be either 'clockwise' or 'counterclockwise'")


        # Loop through all pages and rotate the selected ones
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]

            if page_num in pages_to_rotate:
                page = page.rotate(angle)  # Rotate page by the specified angle

            writer.add_page(page)

        # Write the rotated pages to a new PDF
        with open(output_pdf, 'wb') as output_file:
            writer.write(output_file)

    return "Rotated pages saved as:\n" + output_pdf + " \n\nRefresh the Files area and locate " + output_pdf + "."

### Delete all PDF files from Files

In [41]:
def delete_pdfs():
  """
  Delete all PDF files in the current directory.
  Return a message.
  """

  # Create a list of all PDF files in the current directory
  pdf_files = []
  for filename in os.listdir():
      if filename.endswith('.pdf'):
          pdf_files.append(filename)

  print(pdf_files)

  if len(pdf_files) == 0:
    return "No PDF files found. No action taken."

  # Delete all files in the pdf_files

  for filename in pdf_files:
      os.remove(filename)

  pdf_files_str = ', \n'.join(pdf_files)

  return "Files deleted:\n" + pdf_files_str + "\n\nRefresh the Files area and check that it has no PDF files."

### Run the chosen option

In [42]:
# Run the rotate or delete process depending on the user choice
if rotate_or_delete == "rotate":
  result = rotate_selected_pages(rotated_pdf_path, page_range, direction)
elif rotate_or_delete == "delete":
  result = delete_pdfs()

['Medical Forms - Filled.pdf', 'output.pdf']


## Result

In [43]:
print(result)

Files deleted:
Medical Forms - Filled.pdf, 
output.pdf

Refresh the Files area and check that it has no PDF files.
